Functions and imports


In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
main_copy_dir = os.path.join(parent_dir, "main_copy")
octagon_analysis = os.path.join(parent_dir, "octagon_analysis")
sys.path.append(main_copy_dir)
sys.path.append(octagon_analysis)

print("Added paths to sys.path:", main_copy_dir, octagon_analysis)

Added paths to sys.path: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy /Users/benny/Desktop/MSc/Project/Git/repos/octagon_analysis


Imports with autoreload

In [2]:
%load_ext autoreload
%autoreload 2

import analysis.trajectory_efficiency as trajectory_efficiency
import pandas as pd
import numpy as np
import parse_data.prepare_data as prepare_data
import parse_data.identify_filepaths as identify_filepaths
import analysis.conditioned_player_choice as conditioned_player_choice
import prepare_questionnaire_data
import data_extraction.get_indices as get_indices
import globals

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define data folder and eprsonal data file

In [3]:
data_folder = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files'
personal_data_csv = '/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/personal_data.csv'

Prepare questionnaire data

In [4]:
questionnaire_data_df = prepare_questionnaire_data.preprocess_data()

AQ_scores.csv
GAD_scores.csv
BIS_scores.csv


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/questionnaire_data/prepare_questionnaire_data.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].replace(answers_dict[trait]).astype(int)
/Users/benny

Get social and solo filenames (solos ordered by pseudonym, neither ordered by date)

In [5]:
social_files, ordered_solos = identify_filepaths.get_filenames(data_folder=data_folder)

Separate first and second solos into lists

In [6]:
first_solos = ordered_solos[::2]
second_solos = ordered_solos[1::2]

Create list of tuples for pseudonyms in socials

In [7]:
import re
pseudo_tuples_list = []
experiment_dates_socials = []
for file in social_files:
     # match the session number and the pseudonym string
    match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?_.*?)_Social\.json', file)
    if match:
        session, date, time, pseudonyms = match.groups()
        pseudo_tuples = pseudonyms.split('_')[0::]
        pseudo_tuples_list.append(pseudo_tuples)
        experiment_dates_socials.append(date)

In [8]:
social_dates = {}
opponent_dict = {}

for i, pseudonym in enumerate(pseudo_tuples_list):
    player0 = pseudonym[0]
    player1 = pseudonym[1]

    if player0 not in social_dates:
        social_dates[player0] = []
        opponent_dict[player0] = []
    if player1 not in social_dates:
        social_dates[player1] = []
        opponent_dict[player1] = []

    social_dates[player0] = experiment_dates_socials[i]
    social_dates[player1] = experiment_dates_socials[i]

    opponent_dict[player0] = player1
    opponent_dict[player1] = player0


Create list of pseudonyms for first and second solos (in case they are differently ordered, or some are missing I guess?)

In [9]:
first_solo_pseudo_list = []
second_solo_pseudo_list = []
first_solo_dates = []
second_solo_dates = []
solo_dates = {}

counter = 0
for first_solo, second_solo in zip(first_solos, second_solos):
   # match the session number and the pseudonym string
    first_match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?)_FirstSolo\.json', first_solo)
    second_match = re.search(r'(\d+_\d)[/\\](\d{4}-\d{2}-\d{2})_(\d{2}-\d{2}-\d{2})_(.*?)_SecondSolo\.json', second_solo)

    if first_match:
        session, date, time, pseudonyms = first_match.groups()
        first_pseudo = pseudonyms.split('_')[0::]
        first_solo_pseudo_list.append(first_pseudo)
        first_solo_dates.append(date)

        
    if second_match:
        session, date, time, pseudonyms = second_match.groups()
        second_pseudo = pseudonyms.split('_')[0::]
        second_solo_pseudo_list.append(second_pseudo)
        second_solo_dates.append(date)

for i, pseudonym in enumerate(first_solo_pseudo_list):
    player = pseudonym[0]
    if player not in solo_dates:
        solo_dates[player] = []
    solo_dates[player] = first_solo_dates[i]

In [10]:
len(first_solo_pseudo_list)

99

Remove first solo without corresponding second solo

Prepare data and get trial lists

In [11]:
df_first_solos, first_solos_lists = prepare_data.prepare_data(data_folder, first_solos, combine=False)
df_second_solos, second_solos_lists = prepare_data.prepare_data(data_folder, second_solos, combine=False)
df_socials, social_lists = prepare_data.prepare_data(data_folder, social_files, combine=False)
df_solos, solos_lists = prepare_data.prepare_data(data_folder, ordered_solos, combine=False)

filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250417_1/2025-04-17_12-09-51_CY17_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/250417_1/2025-04-17_12-10-56_RL17_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241112_2/2024-11-12_15-13-34_FA12_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241112_2/2024-11-12_15-11-45_SL12_FirstSolo.json
Loading complete.
Preprocessing complete.
filepath: /Users/benny/Desktop/MSc/Project/Git/repos/main_copy/questionnaire_analysis/pseudonymised_json_files/241210_1/2024-12-10_14-13-47_TE10_FirstSolo.json
Loading complete.
Preprocessing 

In [269]:
len(first_solos), len(second_solos), len(ordered_solos)/2, len(social_files)

(89, 89, 89.0, 45)

Combine solos and trial lists

In [12]:
# combined solos
combined_solos = []

for trial_list_idx in range(0, len(solos_lists), 2):

    print(ordered_solos[trial_list_idx], ordered_solos[trial_list_idx + 1]) 

    # concatenate the trial lists for the 2 solos of this session
    trial_list_combined = solos_lists[trial_list_idx] + solos_lists[trial_list_idx + 1]

    combined_solos.append(trial_list_combined)

250417_1/2025-04-17_12-09-51_CY17_FirstSolo.json 250417_1/2025-04-17_12-46-27_CY17_SecondSolo.json
250417_1/2025-04-17_12-10-56_RL17_FirstSolo.json 250417_1/2025-04-17_12-46-05_RL17_SecondSolo.json
241112_2/2024-11-12_15-13-34_FA12_FirstSolo.json 241112_2/2024-11-12_15-44-54_FA12_SecondSolo.json
241112_2/2024-11-12_15-11-45_SL12_FirstSolo.json 241112_2/2024-11-12_15-45-35_SL12_SecondSolo.json
241210_1/2024-12-10_14-13-47_TE10_FirstSolo.json 241210_1/2024-12-10_14-45-41_TE10_SecondSolo.json
241210_1/2024-12-10_14-12-58_TK10_FirstSolo.json 241210_1/2024-12-10_14-45-09_TK10_SecondSolo.json
250509_1/2025-05-09_12-18-03_AM09_FirstSolo.json 250509_1/2025-05-09_12-53-10_AM09_SecondSolo.json
250509_1/2025-05-09_12-18-53_MC09_FirstSolo.json 250509_1/2025-05-09_12-53-00_MC09_SecondSolo.json
250401_2/2025-04-01_15-17-03_AL01_FirstSolo.json 250401_2/2025-04-01_15-52-21_AL01_SecondSolo.json
250401_2/2025-04-01_15-17-26_NL01_FirstSolo.json 250401_2/2025-04-01_15-52-54_NL01_SecondSolo.json
250319_1/2

Create pseudonym list for combined solos - will be the same as first/second solos without the extra first solo that doesn't have a second solo match
- Now that the latter is removed the pseudonym lists should really all be the same, just need to check

In [13]:
combined_solo_pseudo_list = second_solo_pseudo_list
combined_solo_dates = second_solo_dates

Choice conditioned on visibility, using conditioned_player_choice.py

Solo-social difference needs matching by pseudonym first

P(H|H first vis)
- first solo 
- second solo 
- combined solo 
- social 
- combined solo - social
- wall sep 1 
- wall sep 2 
- wall sep 4 

P(H|L first vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(H|opponent not vis)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

P(win)
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

Score per unit time
- first solo
- second solo
- combined solo
- social
- combined solo - social
- wall sep 1
- wall sep 2
- wall sep 4

- match to pseudonym and store in table


SOLO P(H|H first vis)

filtered to include HighLow trials only

1. First solos
2. Second solos
3. Combined solos

In [14]:

# variables
inferred_choice = False
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = None
solo = True
data_size_cutoff = 6

In [15]:
## First solo datasets
(probabilities2firstsolo,
 numerator2firstsolo,
  denominator2firstsolo) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(first_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 19
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 1.]
And the number of valid trials at this step is 23
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1.]
And the number of valid trials at this step is 10
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1.]
And the number of

In [16]:
## Second solo datasets
(probabilities2secondsolo,
 numerator2secondsolo,
  denominator2secondsolo) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(second_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0.]
And the number of valid trials at this step is 4
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1.
 0. 1.]
And the number of valid trials at this step is 22
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 1.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 19
filter_trials_one_wall_becomes_visible_first - given

In [17]:
## Combined solo datasets
(probabilities2combinedsolo,
 numerator2combinedsolo,
  denominator2combinedsolo) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(combined_solos, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.
 0. 1. 0.]
And the number of valid trials at this step is 23
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 0.
 1. 1. 0. 1.]
And the number of valid trials at this step is 45
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 0. 0. 0. 1. 0. 1.

Filter by wall separations

Wall sep: 45°

In [18]:

# variables
inferred_choice = False
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 1
solo = True
data_size_cutoff = 6

In [277]:
## First solo datasets
(probabilities2firstsolo_sep1,
 numerator2firstsolo_sep1,
  denominator2firstsolo_sep1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(first_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 12
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1.]
And the number of valid trials at this step is 10
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 0. 0. 0. 0. 1.]
And the number of valid trials at this step is 3
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 3
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visi

In [278]:
## Second solo datasets
(probabilities2secondsolo_sep1,
 numerator2secondsolo_sep1,
  denominator2secondsolo_sep1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(second_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0.]
And the number of valid trials at this step is 1
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 7
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1.
 0. 1. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 10
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0.
 1. 1.]
And the number of valid trials at this step is 14
filter_trials_one_wall_becomes_visible_first - given_w

In [19]:
## Combined solo datasets
(probabilities2combinedsolo_sep1,
 numerator2combinedsolo_sep1,
  denominator2combinedsolo_sep1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(combined_solos, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
And the number of valid trials at this step is 13
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 17
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 13
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0.

Wall sep: 90°

In [20]:

# variables
inferred_choice = False
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 2
solo = True
data_size_cutoff = 6

In [281]:
## First solo datasets
(probabilities2firstsolo_sep2,
 numerator2firstsolo_sep2,
  denominator2firstsolo_sep2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(first_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 3
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1.]
And the number of valid trials at this step is 5
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1.]
And the number of valid trials at this step is 6
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 0.]
And the number of valid trials at this step is 1
fewer than 6 trials in the denominator, returning np.nan instead of probability
fi

In [282]:
## Second solo datasets
(probabilities2secondsolo_sep2,
 numerator2secondsolo_sep2,
  denominator2secondsolo_sep2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(second_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 1. 0. 0. 1.]
And the number of valid trials at this step is 2
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1.]
And the number of valid trials at this step is 9
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 1. 0. 0.]
And the number of valid trials at this step is 2
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
And the number of valid trials at this step is 2
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visibl

In [21]:
## Combined solo datasets
(probabilities2combinedsolo_sep2,
 numerator2combinedsolo_sep2,
  denominator2combinedsolo_sep2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(combined_solos, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
And the number of valid trials at this step is 5
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1.
 0. 1. 0. 1.]
And the number of valid trials at this step is 14
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0.]
And the number of valid trials at this step is 8
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
And the number of valid trials at this step is 3
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall

Wall sep: 180°

In [22]:

# variables
inferred_choice = False
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 4
solo = True
data_size_cutoff = 6

In [285]:
## First solo datasets
(probabilities2firstsolo_sep4,
 numerator2firstsolo_sep4,
  denominator2firstsolo_sep4) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(first_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 0. 1. 0. 0.]
And the number of valid trials at this step is 4
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0.]
And the number of valid trials at this step is 8
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0.]
And the number of valid trials at this step is 1
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 1. 1. 0. 0. 1.]
And the number of valid trials at this step is 5
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 0.]
And the 

In [297]:
## Second solo datasets
(probabilities2secondsolo_sep4,
 numerator2secondsolo_sep4,
  denominator2secondsolo_sep4) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(second_solos_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
And the number of valid trials at this step is 1
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
And the number of valid trials at this step is 6
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 7
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
And the number of valid trials at this step is 5
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first -

In [23]:
## Combined solo datasets
(probabilities2combinedsolo_sep4,
 numerator2combinedsolo_sep4,
  denominator2combinedsolo_sep4) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(combined_solos, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)
    

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
And the number of valid trials at this step is 5
fewer than 6 trials in the denominator, returning np.nan instead of probability
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1.]
And the number of valid trials at this step is 14
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 8
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
And the number of valid trials at this step is 10
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 0. 0. 0. 0.

In [300]:
probabilities2firstsolo_sep4, probabilities2secondsolo_sep4, probabilities2combinedsolo_sep4

(array([       nan, 0.875     ,        nan,        nan,        nan,
               nan,        nan,        nan, 1.        ,        nan,
               nan, 0.28571429,        nan,        nan, 0.9       ,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan, 1.        ,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan, 1.        ,        nan, 0.71428571,
               nan, 1.        , 1.        ,        nan, 0.85714286,
               nan, 0.        ,        nan,        nan,        nan,
               nan, 0.85714286,        nan,        nan, 0.875     ,
               nan,        nan,        nan,        nan,        nan,
               nan,        nan,        nan,        nan,        nan,
               nan, 0.5       ,        nan,        nan,        nan,
               nan, 0.375     , 0.88888889,     

SOCIALS

In [24]:

# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = None
solo = False
data_size_cutoff = 6

In [25]:
## All social datasets
(probabilities2social,
 numerator2social,
  denominator2social) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/trajectory_analysis/trajectory_vectors.py:324: RuntimeWarning: invalid value encountered in scalar divide
  cosine_similarity_this_wall = dot_product[wall_num]/(player_to_alcove_vector_norms[wall_num] * player_vector_norm)


filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0.
 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0.
 1. 1. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
And the number of valid trials at this step is 43
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1.]
And the number of valid trials at this step is 26


/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:47: RuntimeWarning: Mean of empty slice
  return np.argmax(np.nanmean(cosine_similarity_trajectory, axis=1))
/Users/benny/Desktop/MSc/Project/Git/repos/main_copy/analysis/loser_inferred_choice.py:149: RuntimeWarning: Mean of empty slice
  highest_alignment_val = np.max(np.nanmean(cosine_similarity_trajectory, axis=1))


filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1. 0.
 1. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 32
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0.
 1. 1. 0.]
And the number of valid trials at this step is 29
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 

Filter by wall sep

Wall sep: 45°

In [26]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 1
solo = False
data_size_cutoff = 6

In [27]:
## All social datasets
(probabilities2social_sep1,
 numerator2social_sep1,
  denominator2social_sep1) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 17
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
And the number of valid trials at this step is 8
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1.
 1. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
And the number of valid trials at this step is 15
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_sessio

Wall sep: 90°

In [28]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 2
solo = False
data_size_cutoff = 6

In [29]:
## All social datasets
(probabilities2social_sep2,
 numerator2social_sep2,
  denominator2social_sep2) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0.
 0. 0. 1.]
And the number of valid trials at this step is 12
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0.
 0. 0.]
And the number of valid trials at this step is 9
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
And the number of valid trials at this step is 7
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 0.]
And the number of valid trials at this step is 7
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 1. 0.
 0. 1. 0.

Wall sep: 180°

In [30]:
# variables
inferred_choice = True
current_fov = 110
chosen_wall_index = 0
visible_wall_index = 1
wall_initial_vis_only = False
other_visible = None
inverse_other_visible = False
wall_sep = 4
solo = False
data_size_cutoff = 6

In [31]:
## All social datasets
(probabilities2social_sep4,
 numerator2social_sep4,
  denominator2social_sep4) = conditioned_player_choice.probability_trial_outcome_given_conditions_all_sessions(social_lists, inferred_choice, current_fov,
                                                            chosen_wall_index=chosen_wall_index, visible_wall_index=visible_wall_index,
                                                            other_visible=other_visible, wall_initial_vis_only=wall_initial_vis_only,
                                                            wall_sep=wall_sep, inverse_other_visible=inverse_other_visible, solo=solo,
                                                            trial_type=globals.HIGH_LOW, data_size_cutoff=data_size_cutoff, debug=False)

filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0.
 0. 1.]
And the number of valid trials at this step is 14
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 1.
 1.]
And the number of valid trials at this step is 9
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0.]
And the number of valid trials at this step is 10
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0.]
And the number of valid trials at this step is 11
filter_trials_one_wall_becomes_visible_first - given_wall_first_visible_session array:
[0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.]
And the n

Pseudonym assignments + storing in dictionaries

In [32]:

combined_solo_low_first_sep_1 = {}
social_low_first_sep_1 = {}

combined_solo_low_first_sep_2 = {}
social_low_first_sep_2 = {}

combined_solo_low_first_sep_4 = {}
social_low_first_sep_4 = {}

first_solo_low_first = {}
second_solo_low_first = {}
combined_solo_low_first = {}
social_low_first = {}

social_low_first_sep_1_den = {}
social_low_first_sep_2_den = {}
social_low_first_sep_4_den = {}
social_low_first_den = {}

combinedsolo_low_first_sep_1_den = {}
combinedsolo_low_first_sep_2_den = {}
combinedsolo_low_first_sep_4_den = {}
combinedsolo_low_first_den = {}

firstsolo_low_first_den = {}
secondsolo_low_first_den = {}

In [33]:
social_suffix_map = {
    'sep_1': {
        'probability': probabilities2social_sep1,
        'denominator': denominator2social_sep1
    },
    'sep_2': {
        'probability': probabilities2social_sep2,
        'denominator': denominator2social_sep2
    },
    'sep_4': {
        'probability': probabilities2social_sep4,
        'denominator': denominator2social_sep4
    },
    'all_seps': {
        'probability': probabilities2social,
        'denominator': denominator2social
    }
}


In [34]:
solo_suffix_map = {
    'sep_1': {
        'probability': probabilities2combinedsolo_sep1,
        'denominator': denominator2combinedsolo_sep1
    },
    'sep_2': {
        'probability': probabilities2combinedsolo_sep2,
        'denominator': denominator2combinedsolo_sep2
    },
    'sep_4': {
        'probability': probabilities2combinedsolo_sep4,
        'denominator': denominator2combinedsolo_sep4
    },
    'all_seps': {
        'probability': probabilities2combinedsolo,
        'denominator': denominator2combinedsolo
    }
}

separated_solo_suffix_map = {
    'first_solo': {
        'probability': probabilities2firstsolo,
        'denominator': denominator2firstsolo
    },
    'second_solo': {
        'probability': probabilities2secondsolo,
        'denominator': denominator2secondsolo
    }
}


In [35]:
social_dict_map = {
    'sep_1': {
        'probability': social_low_first_sep_1,
        'denominator': social_low_first_sep_1_den
    },
    'sep_2': {
        'probability': social_low_first_sep_2,
        'denominator': social_low_first_sep_2_den
    },
    'sep_4': {
        'probability': social_low_first_sep_4,
        'denominator': social_low_first_sep_4_den
    },
    'all_seps': {
        'probability': social_low_first,
        'denominator': social_low_first_den
    }
}

In [36]:
solo_dict_map = {
    'sep_1': {
        'probability': combined_solo_low_first_sep_1,
        'denominator': combinedsolo_low_first_sep_1_den
    },
    'sep_2': {
        'probability': combined_solo_low_first_sep_2,
        'denominator': combinedsolo_low_first_sep_2_den
    },
    'sep_4': {
        'probability': combined_solo_low_first_sep_4,
        'denominator': combinedsolo_low_first_sep_4_den
    },
    'all_seps': {
        'probability': combined_solo_low_first,
        'denominator': combinedsolo_low_first_den
    }
}

separated_solo_dict_map = {
    'first_solo': {
        'probability': first_solo_low_first,
        'denominator': firstsolo_low_first_den
    },
    'second_solo': {
        'probability': second_solo_low_first,
        'denominator': secondsolo_low_first_den
    }
}

In [37]:
for i, (player0, player1) in enumerate(pseudo_tuples_list):
    for player in (player0, player1):
        # Initialize if not already
        for suffix_dict in social_dict_map.values():
            print(suffix_dict)
            for kind in ['probability', 'denominator']:
                if kind not in suffix_dict:
                    suffix_dict[kind] = {}
                if player not in suffix_dict[kind]:
                    suffix_dict[kind][player] = []

    for suffix, data_arrays in social_suffix_map.items():
        social_dict_map[suffix]['probability'][player0] = data_arrays['probability'][i, 0]
        social_dict_map[suffix]['probability'][player1] = data_arrays['probability'][i, 1]
        social_dict_map[suffix]['denominator'][player0] = len(data_arrays['denominator'][0][i])
        social_dict_map[suffix]['denominator'][player1] = len(data_arrays['denominator'][1][i])

{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': []}, 'denominator': {'CY17': []}}
{'probability': {'CY17': np.float64(0.8823529411764706), 'RL17': np.float64(0.625)}, 'denominator': {'CY17': 17, 'RL17': 8}}
{'probability': {'CY17': np.float64(0.6666666666666666), 'RL17': np.float64(0.4444444444444444)}, 'denominator': {'CY17': 12, 'RL17': 9}}
{'probability': {'CY17': np.float64(0.35714285714285715), 'RL17': np.float64(0.3333333333333333)}, 'denominator': {'CY17': 14, 'RL17': 9}}
{'probability': {'CY17': np.float64(0.6511627906976745), 'RL17': np.float64(0.46153846153846156)}, 'denominator': {'CY17': 43, 'RL17': 26}}
{'probability': {'CY17': np.float64(0.8823529411764706), 'RL17': n

In [41]:
for i, pseudonym in enumerate(combined_solo_pseudo_list):
    player = pseudonym[0]
    for solo_suffix_dict in solo_dict_map.values():
        print(solo_suffix_dict)
        for kind in ['probability', 'denominator']:
            if kind not in solo_suffix_dict:
                solo_suffix_dict[kind] = {}
            if player not in solo_suffix_dict[kind]:
                solo_suffix_dict[kind][player] = []

    for suffix, data_arrays in solo_suffix_map.items():
        print(suffix)
        print(data_arrays['probability'][i])
        #for kind in ['probability', 'denominator']:
        solo_dict_map[suffix]['probability'][player] = data_arrays['probability'][i]
        solo_dict_map[suffix]['denominator'][player] = len(data_arrays['denominator'][i])

{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
sep_1
1.0
sep_2
nan
sep_4
nan
all_seps
0.9130434782608695
{'probability': {'CY17': np.float64(1.0)}, 'denominator': {'CY17': 13}}
{'probability': {'CY17': np.float64(nan)}, 'denominator': {'CY17': 5}}
{'probability': {'CY17': np.float64(nan)}, 'denominator': {'CY17': 5}}
{'probability': {'CY17': np.float64(0.9130434782608695)}, 'denominator': {'CY17': 23}}
sep_1
1.0
sep_2
1.0
sep_4
0.9285714285714286
all_seps
0.9777777777777777
{'probability': {'CY17': np.float64(1.0), 'RL17': np.float64(1.0)}, 'denominator': {'CY17': 13, 'RL17': 17}}
{'probability': {'CY17': np.float64(nan), 'RL17': np.float64(1.0)}, 'denominator': {'CY17': 5, 'RL17': 14}}
{'probability': {'CY17': np.float64(nan), 'RL17': np.float64(0.9285714285714286)}, 'denominator': {'CY17': 5, 'RL17': 14}}
{'probability': {'CY17': np.float64(0.9130434782608695), 'RL17': np.floa

In [42]:
solo_dict_map

{'sep_1': {'probability': {'CY17': np.float64(1.0),
   'RL17': np.float64(1.0),
   'FA12': np.float64(0.9230769230769231),
   'SL12': np.float64(1.0),
   'TE10': np.float64(nan),
   'TK10': np.float64(nan),
   'AM09': np.float64(0.8571428571428571),
   'MC09': np.float64(1.0),
   'AL01': np.float64(0.9230769230769231),
   'NL01': np.float64(0.9333333333333333),
   'HL19': np.float64(1.0),
   'AW19': np.float64(0.875),
   'YL15': np.float64(0.875),
   'RR15': np.float64(0.9),
   'SS22': np.float64(1.0),
   'AD22': np.float64(nan),
   'NK13': np.float64(1.0),
   'RD13': np.float64(nan),
   'AV19': np.float64(0.7777777777777778),
   'XG19': np.float64(0.9),
   'SH27': np.float64(0.9),
   'EN27': np.float64(nan),
   'SP17': np.float64(nan),
   'AW17': np.float64(nan),
   'AR10': np.float64(0.9444444444444444),
   'MY10': np.float64(0.8571428571428571),
   'JU19': np.float64(nan),
   'SY19': np.float64(nan),
   'NO12': np.float64(1.0),
   'AS12': np.float64(1.0),
   'IN04': np.float64(1.0),

In [43]:
for i, pseudonym in enumerate(first_solo_pseudo_list):
    player = pseudonym[0]
    for separated_solo_suffix_dict in separated_solo_dict_map.values():
        print(separated_solo_suffix_dict)
        for kind in ['probability', 'denominator']:
            if kind not in separated_solo_suffix_dict:
                separated_solo_suffix_dict[kind] = {}
            if player not in separated_solo_suffix_dict[kind]:
                separated_solo_suffix_dict[kind][player] = []

    for suffix, data_arrays in separated_solo_suffix_map.items():
        print(suffix)
        print(data_arrays['probability'][i])
        #for kind in ['probability', 'denominator']:
        separated_solo_dict_map[suffix]['probability'][player] = data_arrays['probability'][i]
        separated_solo_dict_map[suffix]['denominator'][player] = len(data_arrays['denominator'][i])

{'probability': {}, 'denominator': {}}
{'probability': {}, 'denominator': {}}
first_solo
0.9473684210526315
second_solo
nan
{'probability': {'CY17': np.float64(0.9473684210526315)}, 'denominator': {'CY17': 19}}
{'probability': {'CY17': np.float64(nan)}, 'denominator': {'CY17': 4}}
first_solo
0.9565217391304348
second_solo
1.0
{'probability': {'CY17': np.float64(0.9473684210526315), 'RL17': np.float64(0.9565217391304348)}, 'denominator': {'CY17': 19, 'RL17': 23}}
{'probability': {'CY17': np.float64(nan), 'RL17': np.float64(1.0)}, 'denominator': {'CY17': 4, 'RL17': 22}}
first_solo
0.5
second_solo
0.7894736842105263
{'probability': {'CY17': np.float64(0.9473684210526315), 'RL17': np.float64(0.9565217391304348), 'FA12': np.float64(0.5)}, 'denominator': {'CY17': 19, 'RL17': 23, 'FA12': 10}}
{'probability': {'CY17': np.float64(nan), 'RL17': np.float64(1.0), 'FA12': np.float64(0.7894736842105263)}, 'denominator': {'CY17': 4, 'RL17': 22, 'FA12': 19}}
first_solo
0.7777777777777778
second_solo
1

In [ ]:
df = pd.read_csv('individual_analyses_table.csv', index_col=0)

In [68]:

# --------- 1. Assign probability values ---------
# Define all value sources and target columns
results = {}

for pseudonym in df.index:
    results[pseudonym] = {
        'pH_L_first_vis_combined_solo_sep1': solo_dict_map['sep_1']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_sep2': solo_dict_map['sep_2']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_sep4': solo_dict_map['sep_4']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_all_seps': solo_dict_map['all_seps']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_first_solo_all_seps': separated_solo_dict_map['first_solo']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_second_solo_all_seps': separated_solo_dict_map['second_solo']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep1': social_dict_map['sep_1']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep2': social_dict_map['sep_2']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep4': social_dict_map['sep_4']['probability'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_all_seps': social_dict_map['all_seps']['probability'].get(pseudonym, np.nan)
    }




In [69]:
for pseudonym, value in results.items():
    for key in value.keys():
        df.loc[pseudonym, key] = value[key]

In [80]:

# --------- 2. Compute sensory effects ---------
separators = ['_sep1', '_sep2', '_sep4', '_all_seps']

for pseudonym in df.index:
    for sep in separators:
        # Solo effect
        solo_high = df.loc[pseudonym, f'pH_H_first_vis_combined_solo{sep}']
        solo_low = df.loc[pseudonym, f'pH_L_first_vis_combined_solo{sep}']
        df.loc[pseudonym, f'sensory_effect_solo{sep}'] = solo_high - solo_low

        # Social effect
        social_high = df.loc[pseudonym, f'pH_H_first_vis_social{sep}']
        social_low = df.loc[pseudonym, f'pH_L_first_vis_social{sep}']
        print(pseudonym, sep)
        print(social_high, social_low)
        df.loc[pseudonym, f'sensory_effect_social{sep}'] = social_high - social_low

        # Difference between solo and social
        df.loc[pseudonym, f'solo-social_sensory_effect{sep}'] = (
            df.loc[pseudonym, f'sensory_effect_solo{sep}'] - df.loc[pseudonym, f'sensory_effect_social{sep}']
        )

SP17 _sep1
0.7 0.4444444444444444
SP17 _sep2
0.7 nan
SP17 _sep4
0.8571428571428571 0.1111111111111111
SP17 _all_seps
0.7407407407407407 0.2916666666666667
VC20 _sep1
0.9285714285714286 0.9375
VC20 _sep2
0.8666666666666667 0.375
VC20 _sep4
0.8125 0.23529411764705882
VC20 _all_seps
0.8666666666666667 0.5365853658536586
IN04 _sep1
1.0 1.0
IN04 _sep2
nan nan
IN04 _sep4
nan nan
IN04 _all_seps
1.0 0.7692307692307693
JL19 _sep1
0.9166666666666666 0.38461538461538464
JL19 _sep2
0.7142857142857143 0.375
JL19 _sep4
0.2 0.25
JL19 _all_seps
0.6206896551724138 0.3448275862068966
BC03 _sep1
1.0 nan
BC03 _sep2
0.7142857142857143 nan
BC03 _sep4
0.9333333333333332 0.0
BC03 _all_seps
0.896551724137931 0.0
NM21 _sep1
0.5 0.375
NM21 _sep2
0.8 0.0
NM21 _sep4
1.0 0.0
NM21 _all_seps
0.7333333333333333 0.11538461538461539
XG19 _sep1
0.8333333333333334 0.2
XG19 _sep2
0.7222222222222222 nan
XG19 _sep4
0.4444444444444444 0.2222222222222222
XG19 _all_seps
0.6923076923076923 0.16666666666666666
MY10 _sep1
0.5 0.29

In [129]:
denom_df = pd.read_csv('individual_analyses_trial_counts.csv', index_col=0)

In [131]:

# --------- 1. Assign denominator values ---------
# Define all value sources and target columns
denom_results = {}

for pseudonym in denom_df.index:
    denom_results[pseudonym] = {
        'pH_L_first_vis_combined_solo_sep1': solo_dict_map['sep_1']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_sep2': solo_dict_map['sep_2']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_sep4': solo_dict_map['sep_4']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_combined_solo_all_seps': solo_dict_map['all_seps']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_first_solo_all_seps': separated_solo_dict_map['first_solo']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_second_solo_all_seps': separated_solo_dict_map['second_solo']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep1': social_dict_map['sep_1']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep2': social_dict_map['sep_2']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_sep4': social_dict_map['sep_4']['denominator'].get(pseudonym, np.nan),
        'pH_L_first_vis_social_all_seps': social_dict_map['all_seps']['denominator'].get(pseudonym, np.nan)
    }




In [132]:
for pseudonym, value in denom_results.items():
    for key in value.keys():
        denom_df.loc[pseudonym, key] = value[key]

In [134]:
denom_df.to_csv('individual_analyses_trial_counts_2.csv')

In [100]:
df.to_csv('individual_analyses_table_2.csv')